In [9]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer


model = AutoModelForSequenceClassification.from_pretrained(
    "veliander/autonlp-dta15-32947883", use_auth_token=True)

tokenizer = AutoTokenizer.from_pretrained(
    "veliander/autonlp-dta15-32947883", use_auth_token=True)


In [18]:
import pandas as pd
import torch
import numpy as np

# Load test data
test = pd.read_csv('test.csv')
X_test = test['Transcription']
y_test = test['Type'] == 'Truthful'


inputs = tokenizer(X_test[0], return_tensors="pt")
outputs = model(**inputs)
print(outputs)  

sm = torch.nn.Softmax(dim=1)
right = 0
wrong = 0
for doc, label in zip(X_test, y_test):
    inputs = tokenizer(doc, return_tensors="pt")
    outputs = model(**inputs)
    result = sm(outputs.logits)
    inferred = result[0][0].item() < result[0][1].item()
    if label and inferred:
        right += 1
    else:
        wrong += 1

    print(f"{doc} -> {inferred}")

    print(
        f"{result[0][1].item():.2f} Truthful {result[0][0].item():.2f} Deceptive Label: {label} \n\n")

print(f"Right: {right} Wrong: {wrong}")
print(f"Accuracy: {right/(right+wrong):.2f}")


SequenceClassifierOutput(loss=None, logits=tensor([[-0.1011,  0.4673]], grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)
I like my friend and She’s blond and she’s pretty and she’s funny um we like to drink wine together and watch movies or we like to sometimes like go to Kroger and steal things which is bad probably shouldn’t have said that sorry umm she goes to OU and right now she’s on a plane to Vegas with her mom and were best friends since we were eleven and we did a lot of fun things together  -> True
0.64 Truthful 0.36 Deceptive Label: True 


Well this person that I really don’t like, I can’t even tell you how much I don’t like them. They just get on my nerves, they bug me about everything, they don’t help me with nothing at all. Like, they try to encourage me, but it just seem like they just doubting me and just make me feel all bad about myself. So, I don’t really like the way they treat me, especially the people who I hang around. They don’t like the friends 